# NGC346-MPG-435

Updating the dataset: P203020100000

* There is a star in the LWRS aperture, so the background is oversubtracted. I changed the background regions and re-extracted the spectra.
* LiF 1A is the faintest of the four spectra, despite being the guide channel. But the LiF 1A spectrum is brighter in exposure 3 than in the other exposures, and it just matches the SiC spectra between 1050 and 1060 A. Furthermore, LiF 1B agrees pretty well with an FOS spectrum of the same star.
* So I rescaled everything to match the LiF 1 spectrum Exp 3.

COMMENT File updated 26 January 2023.<br>
COMMENT Star in LWRS aperture caused background to be overestimated.<br>
COMMENT We have re-extracted all spectra using new background regions.<br>
COMMENT All segments have been scaled to match the LiF 1 channel of Exposure 03.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E12

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Function to read a regular FUSE spectral file.

def read_fuse(filename):
    f = fits.open(filename)
    data = f[1].data
    f.close()
    spec = make_spectrum (data)
    return data, spec

In [ ]:
# Read keywords from file header.

filename = 'p203020100000all2ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
f.close() 
    
# Target is in the MDRS aperture.  LiF 1A is the guide channel.

In [ ]:
# A star in the LWRS aperture was included in the background calculation. 
# I have re-defined the background regions and re-extracted all of the spectra.
# Let's look at the results.

lif1a_data, lif1a = read_fuse('P20302010001alif2ttagfcal.fit')
lif1b_data, lif1b = read_fuse('P20302010001blif2ttagfcal.fit')
lif2a_data, lif2a = read_fuse('P20302010002alif2ttagfcal.fit')
lif2b_data, lif2b = read_fuse('P20302010002blif2ttagfcal.fit')
sic1a_data, sic1a = read_fuse('P20302010001asic2ttagfcal.fit')
sic1b_data, sic1b = read_fuse('P20302010001bsic2ttagfcal.fit')
sic2a_data, sic2a = read_fuse('P20302010002asic2ttagfcal.fit')
sic2b_data, sic2b = read_fuse('P20302010002bsic2ttagfcal.fit')

In [ ]:
# Smooth the spectral arrays.

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed, combined spectra.  

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()

# The SiC spectra are brightest, and LiF 1A is the faintest of all.

In [ ]:
# LiF 1A is fainter than the other spectra.  But the count-rate plots show that the
# LiF 1A count rate is highest in Exp 03.  Is it enough to explain the difference?

filename = 'P20302010031alif2ttagfcal.fit'
f = fits.open(filename)
exp03_data = f[1].data 
f.close()
exp03_lif1a = make_spectrum (exp03_data)
exp03_lif1a_bsmooth = box_smooth(exp03_lif1a, width=15)

filename = 'P20302010031blif2ttagfcal.fit'
f = fits.open(filename)
exp03_data = f[1].data 
f.close()
exp03_lif1b = make_spectrum (exp03_data)
exp03_lif1b_bsmooth = box_smooth(exp03_lif1b, width=15)

# Plot the smoothed spectra.  

f, ax = plt.subplots()  
ax.step(exp03_lif1a_bsmooth.spectral_axis, exp03_lif1a_bsmooth.flux, label='EXP 03') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_xlim([1020, 1080])
ax.set_ylim([0,3])

# The spectrum of exposure 3 is as bright as that of SiC 2B between 1050 and 1060 A,
# suggesting that it is correct.

In [ ]:
# To check, we compare with an FOS spectrum of the same star.

filename = 'y14m5403t_c0f.fits'
f = fits.open(filename)
row = f[0].header['NAXIS2'] - 1
data2d = f[0].data
wave = data2d[row,:] * u.AA
f.close()

filename = 'y14m5403t_c1f.fits'
f = fits.open(filename)
print ('Target:', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APER_FOV'])
data2d = f[0].data
flux = data2d[row,:] * norm * u.Unit('erg cm-2 s-1 AA-1')
f.close()

fos = Spectrum1D(spectral_axis=wave, flux=flux)

f, ax = plt.subplots()
ax.plot(fos.spectral_axis, fos.flux, label='FOS') 
ax.step(exp03_lif1b_bsmooth.spectral_axis, exp03_lif1b_bsmooth.flux, label='EXP 03') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B')
ax.legend()
ax.set_xlim([1080,1220])
# ax.set_xlim([1140,1200])

# The FOS spectrum was taken through a small, but not miniscule, slit.  I'm not going to
# scale the FUSE spectrum to match, just use it for guidance.

# Exp 03 matches the FOS spectrum at the longest wavelengths, so we'll use it as our reference.
# LiF 2A matches Exp 03 at short wavelengths, but suffers from the worm longward of 1120 A.
# We rescale LiF 1B to match Exp 03 and assume that LiF 2A is OK.

In [ ]:
# Rescale LiF 1B to match Exp 3.

# Select a spectral region.

region = SpectralRegion(1145*u.AA, 1170*u.AA)
sub_exp03 = extract_region(exp03_lif1b, region)
sub_lif1b = extract_region(lif1b, region)

# Compute ratio of their fluxes to exp03.

mean03 = sub_exp03.mean()
scale_lif1b = mean03/sub_lif1b.mean()
print ('Scale LiF 1B by', scale_lif1b)
scale_lif2a = 1.0

# Scale the spectra.

lif1b *= scale_lif1b
lif1b_bsmooth *= scale_lif1b

In [ ]:
# Rescale the LiF 1A, 2B and SiC 1A, 2B channels to match EXP 03.

# Select a broad spectral region.

region = SpectralRegion(1040*u.AA, 1070*u.AA)
sub_exp03 = extract_region(exp03_lif1a, region)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of their fluxes to exp03.

mean03 = sub_exp03.mean()
scale_lif1a = mean03/sub_lif1a.mean()
print ('Scale LiF 1A by', scale_lif1a)
scale_lif2b = mean03/sub_lif2b.mean()
print ('Scale LiF 2B by', scale_lif2b)

scale_sic1a = mean03/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1a)
scale_sic2b = mean03/sub_sic2b.mean()
print ('Scale SiC 2B by', scale_sic2b)

# Scale the spectra.

lif1a *= scale_lif1a
lif2b *= scale_lif2b
sic1a *= scale_sic1a
sic2b *= scale_sic2b

lif1a_bsmooth *= scale_lif1a
lif2b_bsmooth *= scale_lif2b
sic1a_bsmooth *= scale_sic1a
sic2b_bsmooth *= scale_sic2b

# The SiC channels are scaled down by 5 to 7%, suggesting that they are contaminated by other stars.
# We will assume that such small contamination factors are acceptable.

In [ ]:
# Plot the rescaled spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1040,1070])

# This looks pretty good.

In [ ]:
# What's happening at short wavelengths?

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([910,1010])

# I am not inclined to rescale either of the SiC spectra.

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 1A.

shift_lif2 = compute_shift(lif2b_data, lif1a_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif1a_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif1a_data, 1040, 1070)

print ('Shift LiF 2 by ', shift_lif2, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif2 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif1a_shift = lif1a
lif1b_shift = lif1b
lif2a_shift = Spectrum1D(spectral_axis=lif2a.spectral_axis + shift_lif2 * u.AA, flux=lif2a.flux)
lif2b_shift = Spectrum1D(spectral_axis=lif2b.spectral_axis + shift_lif2 * u.AA, flux=lif2b.flux)
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1060,1070])

In [ ]:
# We assemble the spectrum.  We use LiF 2A only at wavelengths shortward of 1120 A, where the worm kicks in.

filename = 'p203020100000nvo2ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 917.5))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 917.5))
data['flux'][g] = sic1b_data['flux'][h]
data['error'][g] = sic1b_data['error'][h]

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 917.5) & (data['wave'] < 998))
h = np.where((sic2a_wave > 917.5) & (sic2a_wave < 998))
data['flux'][g] = sic2a_data['flux'][h]
data['error'][g] = sic2a_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 998) & (data['wave'] < 1082.5))
h = np.where((lif1a_wave > 998) & (lif1a_wave < 1082.5))
data['flux'][g] = float(scale_lif1a) * lif1a_data['flux'][h]
data['error'][g] = float(scale_lif1a) * lif1a_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1082.5) & (data['wave'] < 1087))
h = np.where((sic1a_wave > 1082.5) & (sic1a_wave < 1087))
data['flux'][g] = float(scale_sic1a) * sic1a_data['flux'][h]
data['error'][g] = float(scale_sic1a) * sic1a_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087) & (data['wave'] < 1120))
h = np.where((lif2a_wave > 1087) & (lif2a_wave < 1120))
data['flux'][g] = float(scale_lif2a) * lif2a_data['flux'][h]
data['error'][g] = float(scale_lif2a) * lif2a_data['error'][h]

# Scale LiF 1B
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1120) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1120) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = float(scale_lif1b) * lif1b_data['flux'][h]
data['error'][g] = float(scale_lif1b) * lif1b_data['error'][h]

hdr['comment'] = ''
hdr['comment'] = 'File updated 26 January 2023.'
hdr['comment'] = 'Star in LWRS aperture caused background to be overestimated.'
hdr['comment'] = 'We have re-extracted all spectra using new background regions.'
hdr['comment'] = 'All segments have been scaled to match the LiF 1 channel of Exposure 03.'

f.writeto('level0_p203020100000nvo2ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'p203020100000nvo2ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_p203020100000nvo2ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.step(new['wave'], new['flux'], label='NEW FLUX')

ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')

ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([-2e-13,4e-12])

In [ ]:
# An examination of the shortest wavelengths reveals that we are still oversubtracting the background,
# though less than before.

f, ax = plt.subplots()  
ax.step(new['wave'], new['flux'], label='NEW FLUX')
ax.step(old['wave'], old['flux'], label='OLD FLUX') 
ax.legend()
ax.set_xlim([902,930])
ax.set_ylim([-5e-14,2e-13])